# Export hourly mean temperatures
- This script is used to export hourly mean ground temperature, canopy air temperature, and ground temperature from simulations;
- Simulations: CNTL, TRAF at FR-Capitole and UK-Manchester. 

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
home_path = '/gws/nopw/j04/duicv/yuansun/'

In [3]:
jja_months = [6, 7, 8]
djf_months = [12, 1, 2]
var_list = ['TRAFFICFLUX', 'TG_U', 'TSA_U', 'TBUILD']
site_list = ['FR-Cap', 'UK-MCR']
date_list = ['2004-02-20-03600', '2022-01-01-03600']

# Delta 

In [5]:
delta_diurnal_mean = []

for i in range(2):  # loop over sites
    site = site_list[i]
    date = date_list[i]
    
    # Store cntl and traffic data separately
    ds_case_hour_dict = {}
    
    for case in ['cntl', 'traffic']:
        ds_case_hour = xr.open_dataset(f'{home_path}0_urban_traffic/archive/{site}_{case}/lnd/hist/{site}_{case}.clm2.h0.{date}.nc')
        ds_case_hour['time'] = pd.to_datetime(ds_case_hour['time'].values).round('h')
        ds_case_hour_dict[case] = ds_case_hour

    for p in range(2):  # loop over Summer/Winter
        season = ['Summer', 'Winter'][p]
        period = [jja_months, djf_months][p]

        # Select seasonal period for both cntl and traffic
        ds_cntl_hour_period = ds_case_hour_dict['cntl'].sel(time=ds_case_hour_dict['cntl']['time.month'].isin(period))
        ds_traffic_hour_period = ds_case_hour_dict['traffic'].sel(time=ds_case_hour_dict['traffic']['time.month'].isin(period))
        
        for var in var_list:
            if var != 'TRAFFICFLUX':
                # Read cntl
                df_cntl_var = ds_cntl_hour_period[var][:,0].to_dataframe().reset_index()
                df_cntl_var['hour'] = df_cntl_var['time'].dt.hour
                if i == 0:
                    df_cntl_var['hour'] = (df_cntl_var['hour'] + 1) % 24
                df_cntl_hourly_mean = df_cntl_var[['hour', var]].groupby(['hour']).mean()
            
                # Read traffic
                df_traffic_var = ds_traffic_hour_period[var][:,0].to_dataframe().reset_index()
                df_traffic_var['hour'] = df_traffic_var['time'].dt.hour
                if i == 0:
                    df_traffic_var['hour'] = (df_traffic_var['hour'] + 1) % 24
                df_traffic_hourly_mean = df_traffic_var[['hour', var]].groupby(['hour']).mean()
            
                # Now calculate traffic - cntl
                df_delta = df_traffic_hourly_mean - df_cntl_hourly_mean
            else:
                # Read traffic flux
                df_traffic_var = ds_traffic_hour_period[var][:, 0].to_dataframe().reset_index()
                df_traffic_var['hour'] = df_traffic_var['time'].dt.hour
                if i == 0: # adjust the FR-Cap hour to match the local time
                    df_traffic_var['hour'] = (df_traffic_var['hour'] + 1) % 24
                df_traffic_hourly_mean = df_traffic_var[['hour', var]].groupby('hour').mean()
                # normalized traffic flux
                df_delta = (df_traffic_hourly_mean - df_traffic_hourly_mean.min()) / (df_traffic_hourly_mean.max() - df_traffic_hourly_mean.min())
            df_delta['hour'] = df_delta.index  # restore hour as column
            df_delta['site'] = site
            df_delta['season'] = season
            df_delta['var'] = var 
            df_delta.rename(columns={var: 'value'}, inplace=True)
            delta_diurnal_mean.append(df_delta)    

# Combine all results
df_delta_diurnal_mean = pd.concat(delta_diurnal_mean, axis=0)
df_delta_diurnal_mean.reset_index(drop=True, inplace=True)

# Save to csv
df_delta_diurnal_mean.to_csv('data_for_figure/delta_temperature_diurnal_mean.csv', index=False)
df_delta_diurnal_mean

,value,hour,site,season,var
0,0.473556,0,FR-Cap,Summer,TRAFFICFLUX
1,0.474415,1,FR-Cap,Summer,TRAFFICFLUX
2,0.157867,2,FR-Cap,Summer,TRAFFICFLUX
3,0.157888,3,FR-Cap,Summer,TRAFFICFLUX
4,0.052694,4,FR-Cap,Summer,TRAFFICFLUX
...,...,...,...,...,...
379,0.000000,19,UK-MCR,Winter,TBUILD
380,0.000000,20,UK-MCR,Winter,TBUILD
381,0.000000,21,UK-MCR,Winter,TBUILD
382,0.000000,22,UK-MCR,Winter,TBUILD


# simulation values

In [ ]:
diurnal_mean = []
for i in range(2):
    site = site_list[i]
    date = date_list[i]
    for case in ['cntl', 'traffic']:
        ds_case_hour = xr.open_dataset(f'{home_path}0_urban_traffic/archive/{site}_{case}/lnd/hist/{site}_{case}.clm2.h0.{date}.nc')
        ds_case_hour['time'] = pd.to_datetime(ds_case_hour['time'].values).round('h')
        for p in range(2):
            season = ['Summer', 'Winter'][p]
            period = [jja_months, djf_months][p]
            ds_case_hour_period = ds_case_hour.sel(time=ds_case_hour['time.month'].isin(period))  
            for var in var_list:    
                if (var == 'TRAFFICFLUX') & (case == 'cntl'):
                    continue
                df_case_var = ds_case_hour_period[var][:,0].to_dataframe().reset_index()
                df_case_var['hour'] = df_case_var['time'].dt.hour
                if i ==0:
                    df_case_var['hour'] = (df_case_var['hour'] + 1) % 24
                df_case_hourly_mean = df_case_var[['hour', var]].groupby(['hour']).mean()
                #if var != 'TRAFFICFLUX':
                    #df_case_hourly_mean[var] = df_case_hourly_mean[var] -273.15    
                min_val = df_case_hourly_mean[var].min()
                max_val = df_case_hourly_mean[var].max()
                #df_case_hourly_mean_normalized = df_case_hourly_mean.copy()
                df_case_hourly_mean_normalized = (df_case_hourly_mean - min_val) / (max_val - min_val)
                df_case_hourly_mean_normalized['site'] = site
                df_case_hourly_mean_normalized['season'] = season
                df_case_hourly_mean_normalized['case'] = case
                df_case_hourly_mean_normalized['var'] = var
                df_case_hourly_mean_normalized.rename(columns={var: 'value'}, inplace=True)
                diurnal_mean.append(df_case_hourly_mean_normalized)

df_dinural_mean = pd.concat(diurnal_mean, axis=0)
df_dinural_mean.reset_index(drop=False, inplace=True)
df_dinural_mean.to_csv('data_for_figure/temperature_diurnal_mean.csv', index=False)
df_dinural_mean 

In [ ]:
diurnal_mean = []
for i in range(2):
    site = site_list[i]
    date = date_list[i]
    for case in ['cntl', 'traffic']:
        ds_case_hour = xr.open_dataset(f'{home_path}0_urban_traffic/archive/{site}_{case}/lnd/hist/{site}_{case}.clm2.h0.{date}.nc')
        ds_case_hour['time'] = pd.to_datetime(ds_case_hour['time'].values).round('h')
        for p in range(2):
            season = ['Summer', 'Winter'][p]
            period = [jja_months, djf_months][p]
            ds_case_hour_period = ds_case_hour.sel(time=ds_case_hour['time.month'].isin(period))  
            for var in var_list:    
                if (var == 'TRAFFICFLUX') & (case == 'cntl'):
                    continue
                df_case_var = ds_case_hour_period[var][:,0].to_dataframe().reset_index()
                df_case_var['hour'] = df_case_var['time'].dt.hour
                if i ==0:
                    df_case_var['hour'] = (df_case_var['hour'] + 1) % 24
                df_case_hourly_mean = df_case_var[['hour', var]].groupby(['hour']).mean()
                if var != 'TRAFFICFLUX':
                    df_case_hourly_mean[var] = df_case_hourly_mean[var] -273.15    
                #min_val = df_case_hourly_mean[var].min()
                #max_val = df_case_hourly_mean[var].max()
                df_case_hourly_mean_normalized = df_case_hourly_mean.copy()
                #df_case_hourly_mean_normalized = (df_case_hourly_mean - min_val) / (max_val - min_val)
                df_case_hourly_mean_normalized['site'] = site
                df_case_hourly_mean_normalized['season'] = season
                df_case_hourly_mean_normalized['case'] = case
                df_case_hourly_mean_normalized['var'] = var
                df_case_hourly_mean_normalized.rename(columns={var: 'value'}, inplace=True)
                diurnal_mean.append(df_case_hourly_mean_normalized)

df_dinural_mean = pd.concat(diurnal_mean, axis=0)
df_dinural_mean.reset_index(drop=False, inplace=True)
df_dinural_mean.to_csv('data_for_figure/temperature_diurnal.csv', index=False)
df_dinural_mean 